In [2]:
import pandas as pd
df = pd.read_csv("all_sequences.csv")
df.head()


,Header,Sequence
0,6f6f_A|query,MAHHHHHHVDDDDKMVHHDGFQTVKATIDWEHPMFKLYEKAKRNGK...
1,6f6f_A|tr|A0A2V7RGW7|A0A2V7RGW7_9BACT Uncharac...,-----------------GRNI----DGV-PDSVYWKLYFAARK-LP...
2,6f6f_A|tr|A0A2A5EFJ8|A0A2A5EFJ8_9BACT Uncharac...,-------------------------MALNLDSFPLKEIRRGRE-LQ...
3,6f6f_A|LakWasM115_HOW13_FD_contig_21_489690_le...,-------------MGIIMRQNFKtTSEGLNRKHVMLKLWEKAKKLG...
4,6f6f_A|GraSoiStandDraft_60_1057301.scaffolds.f...,------------MEYQK---AVVfADGRLEKTLGPGRYFLT-----...


In [2]:
import numpy as np

def mask_column_MSA_prot(prot_sequences, index):
    """
    Masks a chosen residue of all aligned sequences of a protein. 

    Parameters
    ----------
    prot_sequences : DataFrame
        DataFrame of the sequences of the protein in which we mask a column.
    index:
        Position to mask.

    Returns
    -------
    DataFrame
        DataFrame with the masked position for all sequences.
    """
    if index < len(prot_sequences['Sequence'].iloc[0]):
        prot_sequences['Sequence'] = prot_sequences['Sequence'].apply(lambda seq: seq[:index] + '<mask>' + seq[index+1:])
    else: print('The index indicated does not exist in these sequences')

    return prot_sequences


def mask_column_MSA(input_data):
    """
    Masks a randomly chosen position in all aligned sequences of the proteins in the given file. 

    Parameters
    ----------
    input_data : str or DataFrame
        Either the file path of the proteins MSA file to mask, or a DataFrame containing the MSA.

    Returns
    -------
    DataFrame
        DataFrame of the MSA with the randomly chosen masked position for each protein.
    """
    if isinstance(input_data, str):
        df = pd.read_csv(input_data)
    elif isinstance(input_data, pd.DataFrame):
        df = input_data
    else:
        raise ValueError("Input must be a file path (str) or a DataFrame.")

    # Group by the first 6 characters of the "Header", the anme of the protein
    grouped_dfs = df.groupby(df['Header'].str[:6], sort=False)

    df_new=pd.DataFrame()
    for _, group in grouped_dfs:
        index = np.random.randint(len(group['Sequence'].iloc[0]))
        prot_df=mask_column_MSA_prot(group, index)
        df_new=pd.concat([df_new, prot_df], ignore_index=True)

    return df_new


df=mask_column_MSA(df)



In [3]:
print(df['Header'].iloc[0])
print(df['Header'].iloc[1])
print(df['Header'].iloc[100])
print(df['Header'].iloc[101])
print(df['Header'].iloc[4000])
print(df['Header'].iloc[4001])
print(df['Sequence'].iloc[0])
print(df['Sequence'].iloc[1])
print(df['Sequence'].iloc[100])
print(df['Sequence'].iloc[101])
print(df['Sequence'].iloc[4000])
print(df['Sequence'].iloc[4001])


6f6f_A|query
6f6f_A|tr|A0A2V7RGW7|A0A2V7RGW7_9BACT Uncharacterized protein OS=Gemmatimonadetes bacterium OX=2026742 GN=DMD37_12880 PE=4 SV=1
6f6f_A|SRR5712692_5841859
6f6f_A|SRR5438093_2628182
6e9m_A|UniRef100_A0A4Y7SGG8 Hydrophobin n=1 Tax=Coprinellus micaceus TaxID=71717 RepID=A0A4Y7SGG8_9AGAR
6e9m_A|UniRef100_A0A4S4LPW1 Uncharacterized protein n=1 Tax=Bondarzewia mesenterica TaxID=1095465 RepID=A0A4S4LPW1_9AGAM
MAHHHHHHVDDDDKMVHHDGFQTVKATIDWEHPMFKLYEKAKRNGKWNPADIDFSQDQKDFASLTSEEKISALPL<mask>AGFSAGEEAITLDILPMAHALARQGRLEDVLFLTTFMHDEAKHVEMFSRWQQAVGIGQMDLSVFHNDHYKRIFYEALPEAMNRLYADDSPEAVIRAATVYNMIVEGTLAESGYYTFRQIYKKAGLFPGLLQGIDYLNMDEGRHIQFGIYTIQRIVNEDERYYELFIRYMDELWPHVIGYVDYLTELGKRQQQLARTYALEIDYDLLRHYVIKQFNLRKKQISRTKRVDVVEGLEKTAAES
-----------------GRNI----DGV-PDSVYWKLYFAARK-LPWDPQTVDLSRECADWERIRREfpaedYAG<mask>VLQLCTFFYDGEESVTYTLTPYLSAAArLGLGADLQLFLTSQLAEEARHFEFFRRYFAEVLPD----ANPrmpLPPEPRSVLVDGLQDIADRLRKEDEPGqlreLLVEGVAHYMGVVESMLARTGYRGVGEVLAARGWLPGLQEGFRLIRRDEGRHVAFGIHFLKDIASPhPPLr--KI

In [ ]:
import re
import pandas as pd
import random
import glob
import os

def load_hhr_files(root_dir, output_csv, num_random_folders):
    """
    Load hhr files from a random subset of protein subdirectories and save the sequences to a CSV file.

    Parameters
    ----------
    root_dir : str
        Root directory where protein subdirectories and hhr files are located.
    output_csv : str
        Path to save the CSV file with sequences.
    num_random_folders : int
        Number of random protein folders to select.

    Returns
    -------
    None
    """
    all_struct = []
    print(f"Loading hhr files from {root_dir}")

    # Get all the protein folders
    protein_dirs = [protein_dir for protein_dir in glob.glob(os.path.join(root_dir, '*'), recursive=False) if os.path.isdir(protein_dir)]
    
    # Randomly select num_random_folders protein folders
    selected_proteins = random.sample(protein_dirs, min(num_random_folders, len(protein_dirs)))

    # Process the selected protein folders
    for protein_dir in selected_proteins:
        protein_name = os.path.basename(protein_dir)  # Assign the name of the dir to the sequences
        
        # Find the target hhr file in the subdirectory
        for file_path in glob.glob(os.path.join(protein_dir, 'hhr', '*hhr')):
            print(f"Processing file: {file_path}")
            structures = parse_hhr_file(file_path)
            all_struct.extend(struct)  # Add the parsed data from each file

            #TO MODIFY ##########################################
            for header, struct in structures.items():
                all_struct.append({
                    "Header": f"{protein_name}|{header}",
                    "Sequence": sequence
                })

            ##########################################

    # Save the collected data to CSV
    df = pd.DataFrame(all_struct)  # Convert list of dictionaries to DataFrame
    df.to_csv(output_csv, index=False)


def parse_hhr_file(file_path):
    """
    Parses an hhr file.

    Parameters
    ----------
    file_path : str
        Path to the hhr file.

    Returns
    -------
    list
        List of dictionaries with the parsed data from the file.
    """
    with open(file_path, 'r') as file:
        hhr_output = file.read()

    # Split file into individual HHR blocks
    hhr_blocks = re.split(r"\n>[^>]*", hhr_output)  # Split by header lines that start with ">"
    
    parsed_data = []

    # Loop through each block and parse
    for block in hhr_blocks:
        if block.strip():  # Ignore empty blocks
            parsed_data.append(parse_hhr_block(block))

    return parsed_data

def parse_hhr_block(hhr_block):
    """
    Parse an individual HHR block to extract relevant information.

    Parameters
    ----------
    hhr_block : str
        The raw block content from an HHR file.

    Returns
    -------
    dict
        Dictionary with header, target sequence, ss_dssp, ss_pred, and confidence.
    """
    header_pattern = r"^>(.*?)\n"  # header starting with '>'
    target_sequence_pattern = r"T\s([^\s]+)\s+(?!Consensus|ss_dssp|ss_pred)(.*?)\n"  # target sequence starting with 'T'
    ss_dssp_pattern = r"T ss_dssp\s+(.*?)\n"  # secondary structure dssp line
    ss_pred_pattern = r"T ss_pred\s+(.*?)\n"  # secondary structure prediction line
    confidence_pattern = r"Confidence\s+(.*?)\n"  # Extract the confidence line
    
    header = re.search(header_pattern, hhr_block, re.MULTILINE)
    target_sequence = re.findall(target_sequence_pattern, hhr_block)
    ss_dssp = re.search(ss_dssp_pattern, hhr_block)
    ss_pred = re.search(ss_pred_pattern, hhr_block)
    confidence = re.search(confidence_pattern, hhr_block)

    result = {
        "header": header.group(1) if header else None,
        "target_sequence": target_sequence[0][1] if target_sequence else None,  # Extract the actual sequence part
        "ss_dssp": ss_dssp.group(1) if ss_dssp else None,
        "ss_pred": ss_pred.group(1) if ss_pred else None,
        "confidence": confidence.group(1) if confidence else None,
    }
    return result
